# Accessing texts from NB digital

Texts are accessed and analysed using tools built in the programming language Python. The problem is how to analyse texts when they are protected from distribution. One solution is to not challenge the copyright. Instead features of texts can be downloaded and used for analysis.

In [1]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import dhlab.graph_networkx_louvain as gnl
%matplotlib inline
mu.css()

In [38]:
mu.update('collocations', silent=True)
import collocations as col
import pandas as pd

# Find texts and search in them

Use properties of texts to constrain number of possibilities. Retrieve a selection of books that satisfies those contrainst. This is your corpus. The command is `book_corpus()`, which takes a number of parameters. Corpora can be joined, subtracted and combined in a number of ways. 

An output from a search `nb.book_corpus(author='Røssaak%Eivin%')`

In [3]:
corpus = nb.book_corpus(author='Røssaak%Eivin%')
corpus

,urn,author,title,year
0,2008081400081,"Røssaak, Eivind",Det postmoderne og de intellektuelle,1998
1,2011032206046,"Røssaak, Eivind",(Sic),2001
2,2008061800060,"Røssaak, Eivind","Skrujern, kanoner og trojanske hester",1996
3,2014072405078,"Røssaak, Eivind",Selviakttakelse,2005


This corpus can be searched, for example with the command 

`nb.urn_concordance(word=['postmodernisme', 'postmodernismen'], urns=corpus, before=10, after=10)`

In [7]:
nb.urn_concordance(word=['postmodernisme', 'postmodernismen'], urns=corpus, before=10, after=10)

"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",et essay av Derrida som skyteskive og bevis for at,postmodernisme,og dekonstruksjon er galskap . I en nærlesning av det
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998","distinksjoner : « postmodernitet » brukes om en samfunnstilstand, «",postmodernisme,"» om spesielle retninger innen kunst , litteratur , arkitektur"
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",« post war » litteratur i negativ forstand som «,postmodernisme,» . 6 De sammenligner det nye med sin egen
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998","ny teori , og i USA kalles begge deler «",postmodernisme,» . Denne tilstandens første affirmative teoretikere var de kvinnelige
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",den første ( 1971 ) som bruker selve ordet «,postmodernisme,"» på det nye i positiv forstand , først og"
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",To store 68 - er-grupperinger har reagert svært negativt på,postmodernismen,. De er enten maoister eller maoistenes motstandere på venstresiden
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",konservativ reaksjon mot en ny fiende ; en karrikatur av,postmodernismen,. Jeg har forsøkt å finne noen av de rommene
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",De postmoderne ville « korrigere modernismens eksesser » . Etter,postmodernismen,"kom , ifølge Enrico , en « ultramodernismo » som"
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",definert identitet og en eksistensiell trygghet . For Bell oppstår,postmodernismen,der den « dekadente » modernismen ( det kan her
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",som forsvarer moderniteten mot dem han kaller nykonservative . Under,postmodernismen,"ønsker disse , mener han , å frigjøre seg fra"
"(Sic), Røssaak, Eivind, 2001","arbeidsdeling mellom litteraturviter og forfatter , kan man si at",postmodernismen,"tilsvarer et sammenbrudd i arbeidsfordelingen , litteraturviteren er blitt forfatter"


What is connected with the term *postmodern* in the digital collection? Let us start with a larger corpus, one that contains one version of the word or the other

In [16]:
capitalize = lambda words: [x.capitalize() for x in words]

In [18]:
postmodern_words = "postmodernisme postmodernismen postmoderne".split()
postmodern_words += capitalize(postmodern_words)
postmodern_words

['postmodernisme',
 'postmodernismen',
 'postmoderne',
 'Postmodernisme',
 'Postmodernismen',
 'Postmoderne']

In [22]:
postmodern = nb.book_corpus(words= postmodern_words, limit = 20000)


In [26]:
mu.printmd('Number of books satisfying the postmodern condition: ' + "**" + str(len(postmodern))+ "**.")
postmodern.head()

Number of books satisfying the postmodern condition: **8906**.

,urn,author,title,year
0,2016062348061,"Gaarder, Jostein",Maya,2001
1,2010080208038,,Innvandringspolitikk og utlendingsloven,1998
2,2008010304101,,God eksamen,1994
3,2013061905026,,Folkekirke i forandring,2010
4,2008111800114,"Jordheim, Helge",Lesningens vitenskap,2001


In [32]:
speedkonk = lambda søkeord, input_korpus, before=10, after=10: nb.urn_concordance(word=søkeord, urns=nb.refine_book_urn(urns=nb.pure_urn(input_korpus), words=søkeord, limit=10), after=after, before=before,)

In [33]:
speedkonk('litteratur', postmodern)

"Vennskap mellom menn på 1800-tallet, Storberg, Marianne, 2009","oppvurdering av mannlige vennskap kommer til uttrykk i filosofi ,",litteratur,og kunst . 3 Mannlige forhold kunne i praksis bety
"Vennskap mellom menn på 1800-tallet, Storberg, Marianne, 2009",", som har satt sitt tydelige preg på kunst og",litteratur,", farget også mellommenneskelige relasjoner . Mannlige vennskap i denne"
"Vennskap mellom menn på 1800-tallet, Storberg, Marianne, 2009",Thorleif Dahls kulturbibliotek og Det norske akademi for sprog og,litteratur,: 75.
"Vennskap mellom menn på 1800-tallet, Storberg, Marianne, 2009","det ble mange blindspor . Gjennom litteraturen , særlig biografisk",litteratur,"om personer som levde på 1800 - tallet , kunne"
"Vennskap mellom menn på 1800-tallet, Storberg, Marianne, 2009",de biografiske fremstillingene til hvert kapittel støtter jeg meg på,litteratur,av eldre og nyere dato . Behandlingen av vennskapene er
"Medievalgkamp, Eide, Martin, 1991","beskrives hos Habermas , Sennett og Meyrowitz tegnet i amerikansk",litteratur,"og debatt om moderne valgkamper. Marketing-teknikker , meningsmålinger — og"
"Medievalgkamp, Eide, Martin, 1991",1 Når det her refereres til amerikansk,litteratur,"om partienes reduserte rolle , er det viktig å huske"
"Medievalgkamp, Eide, Martin, 1991",klassiske effektforskningen; det andre har først senere resultert i omfattende,litteratur,.
"Barn og TV-reklame, Isachsen, Terje, 1995",Denne rapporten er en litteraturstudie av,litteratur,som tar opp forholdet mellom barn og TV-reklame og forholdet
"Barn og TV-reklame, Isachsen, Terje, 1995",. Takk også til Ragnhild Bjørnebekk for gode forslag til,litteratur,.
"Barn og TV-reklame, Isachsen, Terje, 1995",1.1.2 Ordning av,litteratur,"etter begrepene "" sender "" , "" budskap "" ,"


In [37]:
pm = dict()
urnlist = nb.pure_urn(postmodern)
step = 500
for i in range(0, len(postmodern), step):
    pm[i] = col.dist_coll_urn('postmodernismen',urns=urnlist[i:i+step], before=0, after=10)

In [53]:
pm_tot = pd.concat([pm[i] for i in pm])

In [55]:
pm_tot.loc['han']

,freq,dist,score
han,87.0,116.15,111.31
han,5.0,7.80,7.34
han,5.0,6.00,5.90
han,3.0,4.67,4.95
han,8.0,7.00,6.81
han,2.0,8.00,6.75
han,8.0,7.80,7.51
han,1.0,6.00,5.50
han,5.0,7.00,6.70
han,5.0,5.60,5.58
